## Tide gauge data processing
#### 1. Convert tide stations data (e.g., .txt file from Marine Department) to .nc file
#### 3. Merge the tgauge stations data from Marine Department and Obversatory, respectively

## Tide stations data (.txt) from **Marine Department**

In [2]:
import sys
sys.path.append("/Users/luo/OneDrive/SAR-Altimetry/sentinel3-altimetry-l2/utils")
from get_filesPath import get_filesPath
import numpy as np
import xarray as xr


### Postition of the tide stations
lat_cc, lon_cc = np.array([22.214]), np.array([114.022])  
lat_klw, lon_klw = np.array([22.459]), np.array([114.361])  
lat_kc, lon_kc = np.array([22.324]), np.array([114.122])  
lat_mw, lon_mw = np.array([22.364]), np.array([114.071])  

### Super-parameter

In [7]:
root_dir = os.path.dirname(os.getcwd())
name = ['mw']
long_name = 'Ma wan from Hong Kong Marine Department'
lat, lon = np.array([22.364]), np.array([114.071])
dir_md_tgauge = root_dir+'/data/tgauge_data/marine_department_hk/ma_wan(mw)'
pat_to_save = dir_md_tgauge+'/tgauge_mw.nc'
print(dir_md_tgauge)


/Users/luo/OneDrive/SAR-Altimetry/sentinel3-altimetry-l2/data/tgauge_data/marine_department_hk/ma_wan(mw)


### Extract the time and sea level from the tige files

In [8]:
## get files path name
files_path = get_filesPath(dir_md_tgauge, key_words='.txt')
files_path = sorted(files_path)
print('.txt files number:', len(files_path))
times = []
sea_level = []
for file_path in files_path:
    f = open(file_path,"r")
    text = f.read()
    text = text.split('\n')
    for i in range(len(text)-1):
        time_i = np.datetime64(text[i][0:4]+'-'+text[i][5:7]+'-'+text[i][8:19])
        time_i_utc = time_i - np.timedelta64(8, 'h')
        sea_level_i = float(text[i][20:])
        times.append(time_i_utc)
        sea_level.append(sea_level_i)

print(len(times))
print(len(sea_level))


.txt files number: 1827
263088
263088


'31 23:50:00'

### Save as the xarray format

In [9]:
dset_md = xr.Dataset({
    'sea_level': xr.DataArray(
        data = sea_level,
        dims = ['time'],
        coords = {'time': times},
    ),
    'lat': xr.DataArray(
        data = lat,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'lon': xr.DataArray(
        data = lon,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'station_name': xr.DataArray(
        data = name,
        dims= 'record_id',
        coords = {'record_id': name},
        attrs={'long_name': long_name}
    )})
dset_md['time']


<xarray.DataArray 'time' (time: 263088)>
array(['2015-12-31T16:00:00.000000000', '2015-12-31T16:10:00.000000000',
       '2015-12-31T16:20:00.000000000', ..., '2020-12-31T15:30:00.000000000',
       '2020-12-31T15:40:00.000000000', '2020-12-31T15:50:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2015-12-31T16:00:00 ... 2020-12-31T15:50:00

#### Save the tgauge data

In [10]:
dset_md.to_netcdf(path=pat_to_save)


### Merge all the tgauge stations data from Marine Department

In [32]:
path_md_cc = root_dir+'/data/tgauge_data/marine_department_hk/cheung_chau(cc)/tgauge_cc.nc'
path_md_klw = root_dir+'/data/tgauge_data/marine_department_hk/ko_lau_wan(klw)/tgauge_klw.nc'
path_md_kc = root_dir+'/data/tgauge_data/marine_department_hk/kwai_chung(kc)/tgauge_kc.nc'
path_md_mw = root_dir+'/data/tgauge_data/marine_department_hk/ma_wan(mw)/tgauge_mw.nc'
path_md_save = root_dir+'/data/tgauge_data/marine_department_hk/tgauge_md.nc'

In [33]:
tgauge_cc = xr.open_dataset(path_md_cc)
tgauge_klw = xr.open_dataset(path_md_klw)
tgauge_kc = xr.open_dataset(path_md_kc)
tgauge_mw = xr.open_dataset(path_md_mw)
tgauge_md = xr.concat([tgauge_cc, tgauge_klw, tgauge_kc, tgauge_mw], dim="record_id")
tgauge_md['station_name'].attrs['long_name'] = 'Tide gauge data from Hong Kong Marine Department'
tgauge_md['time']

<xarray.DataArray 'time' (time: 263088)>
array(['2015-12-31T16:00:00.000000000', '2015-12-31T16:10:00.000000000',
       '2015-12-31T16:20:00.000000000', ..., '2020-12-31T15:30:00.000000000',
       '2020-12-31T15:40:00.000000000', '2020-12-31T15:50:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2015-12-31T16:00:00 ... 2020-12-31T15:50:00

In [35]:
# tgauge_md.to_netcdf(path=path_md_save)


## Tide stations data from Hong Kong Observatory


### Postition of the tide stations
lat_qb, lon_qb = np.array([22.3]), np.array([114.217])  
lat_sp, lon_sp = np.array([22.220]), np.array([113.894])  
lat_tmw, lon_tmw = np.array([22.270]), np.array([114.289])  
lat_tpk, lon_tpk = np.array([22.443]), np.array([114.184])  
lat_tbt, lon_tbt = np.array([22.487]), np.array([114.014])  

In [24]:
dir_obser_tgauge = root_dir+'/data/tgauge_data/observatory_hk/tsim_bei_tsui(tbt)'
path_obser_save = dir_obser_tgauge+'/tgauge_obser_tbt.nc'
name = ['tbt']
long_name = 'Tsim bei tsui from Hong Kong Oberatory'
lat, lon = np.array([22.487]), np.array([114.014])


### Read sea level value the corresponding time

In [25]:
## get files path name
files_path = get_filesPath(dir_obser_tgauge, key_words='20')
files_path = sorted(files_path)
print('files number:', len(files_path))
times, sea_level = [], []
for file_path in files_path:
    f = open(file_path,"r")
    text = f.read().split('\n')
    year = file_path[-4:]
    for day_ind in range(len(text)-1):
        month = text[day_ind][0:2].replace(" ","").zfill(2)
        day = text[day_ind][2:4]
        for hour_ind in range(1, 25):
            time = str(hour_ind).zfill(2)+':'+'00'
            if hour_ind < 24:
                time_i = np.datetime64(year+'-' + month + '-'+day + ' ' + time)
            else:               
                time_i = time_i + np.timedelta64(1, 'h')
            time_i_utc = time_i - np.timedelta64(8, 'h')  # convert to utc time
            sea_level_i = float(text[day_ind][hour_ind*4:hour_ind*4+4])/100
            times.append(time_i_utc)
            sea_level.append(sea_level_i)

print(times[-10:])
print(sea_level[-10:])
# sea_level[339:352]


files number: 5
[numpy.datetime64('2020-12-31T07:00'), numpy.datetime64('2020-12-31T08:00'), numpy.datetime64('2020-12-31T09:00'), numpy.datetime64('2020-12-31T10:00'), numpy.datetime64('2020-12-31T11:00'), numpy.datetime64('2020-12-31T12:00'), numpy.datetime64('2020-12-31T13:00'), numpy.datetime64('2020-12-31T14:00'), numpy.datetime64('2020-12-31T15:00'), numpy.datetime64('2020-12-31T16:00')]
[1.6, 1.45, 1.44, 1.69, 2.03, 2.4, 2.72, 2.87, 2.92, 2.85]


### Save to .nc file

In [26]:
dset_obser = xr.Dataset({
    'sea_level': xr.DataArray(
        data = sea_level,
        dims = ['time'],
        coords = {'time': times},
    ),
    'lat': xr.DataArray(
        data = lat,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'lon': xr.DataArray(
        data = lon,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'station_name': xr.DataArray(
        data = name,
        dims= 'record_id',
        coords = {'record_id': name},
        attrs={'long_name': long_name}
    )})
dset_obser
dset_obser.to_netcdf(path=path_obser_save)


In [27]:
path_obser_qb = root_dir+'/data/tgauge_data/observatory_hk/quarry_bay(qb)/tgauge_obser_qb.nc'
path_obser_sp = root_dir+'/data/tgauge_data/observatory_hk/shek_pik(sp)/tgauge_obser_sp.nc'
path_obser_tmw = root_dir+'/data/tgauge_data/observatory_hk/tai_miu_wan(tmw)/tgauge_obser_tmw.nc'
path_obser_tpk = root_dir+'/data/tgauge_data/observatory_hk/tai_po_kau(tpk)/tgauge_obser_tpk.nc'
path_obser_tsui = root_dir+'/data/tgauge_data/observatory_hk/tsim_bei_tsui(tbt)/tgauge_obser_tbt.nc'
path_obser_save = root_dir+'/data/tgauge_data/observatory_hk/tgauge_obser.nc'


In [28]:
tgauge_qb = xr.open_dataset(path_obser_qb)
tgauge_sp = xr.open_dataset(path_obser_sp)
tgauge_tmw = xr.open_dataset(path_obser_tmw)
tgauge_tpk = xr.open_dataset(path_obser_tpk)
tgauge_tsui = xr.open_dataset(path_obser_tsui)

tgauge_obser = xr.concat([tgauge_qb, tgauge_sp, tgauge_tmw, tgauge_tpk, tgauge_tsui], dim="record_id")
tgauge_obser['station_name'].attrs['long_name'] = 'Tide gauge data from Hong Kong Observatory'
tgauge_obser

<xarray.Dataset>
Dimensions:       (record_id: 5, time: 43848)
Coordinates:
  * time          (time) datetime64[ns] 2015-12-31T17:00:00 ... 2020-12-31T16...
  * record_id     (record_id) object 'qb' 'sp' 'tmw' 'tpk' 'tbt'
Data variables:
    sea_level     (record_id, time) float64 2.01 2.01 1.87 ... 2.87 2.92 2.85
    lat           (record_id) float64 22.3 22.22 22.27 22.44 22.49
    lon           (record_id) float64 114.2 113.9 114.3 114.2 114.0
    station_name  (record_id) object 'qb' 'sp' 'tmw' 'tpk' 'tbt'

In [30]:
tgauge_obser.to_netcdf(path=path_obser_save)
